<a href="https://colab.research.google.com/github/RiseAboveAll/RSNA_Pneumonia/blob/master/Resnet50_SSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 1.2MB/s 


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

import pydicom

import csv
import random

from glob import glob #The glob module finds all the pathnames matching a specified pattern according to the rules
import os

import tensorflow as tf
#from tensorflow import keras

from skimage import measure
from skimage.transform import resize
from PIL import Image
from sklearn.model_selection import train_test_split as tts

import math
import cv2

#from tensorflow.keras import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import array_to_img

In [ ]:
import graphviz
import pydotplus
from keras.utils import plot_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Path='/content/drive/My Drive/'
Image_Path='/content/drive/My Drive/Capstone/train_images/'
os.chdir(Path)

In [ ]:
train_X=pd.read_csv('/content/drive/My Drive/Train_X.csv')
val_X=pd.read_csv('/content/drive/My Drive/Val_X.csv')

In [ ]:

IMAGE_SIZE = 224 # Image sizes can vary (128, 160, 192, 224)

EPOCHS = 15 
BATCH_SIZE = 32 
PATIENCE = 10 # Patience for early stopping

MULTI_PROCESSING = False 
THREADS = 1 


In [ ]:
#msk= np.zeros(img.shape)
class DataSequence(tf.keras.utils.Sequence):

    def __init__(self, data):
        self.paths = []
        #self.mask=[]
        #self.msk= np.zeros(img.shape)
        self.coords=np.zeros((data.shape[0],4))
        for index,row in data.iterrows():
            for i,r in enumerate(row[1:5]):
                row[i+1]=int(r)
            path,xmin,xmax,ymin,ymax,_=row
            #msk[ymin:ymax,xmin:xmax]=1
            #self.mask.append(msk)
            self.coords[index,0]=xmin/IMAGE_SIZE
            self.coords[index,1]=ymin/IMAGE_SIZE
            self.coords[index,2]=xmax/IMAGE_SIZE
            self.coords[index,3]=ymax/IMAGE_SIZE
            #self.coords[index,4]=Target
            self.paths.append(path)
        

        
    def __len__(self):
        return math.ceil(len(self.coords) / BATCH_SIZE)

    def __getitem__(self, idx): # Get a batch
        batch_paths = self.paths[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE] # Image path
        batch_coords = self.coords[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE] # Image coords

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE,3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            #img = Image.open(f) # Read image
            img = pydicom.dcmread(f).pixel_array
            img = np.expand_dims(img, -1)
            img=array_to_img(img)
            img=img.resize((IMAGE_SIZE, IMAGE_SIZE))
            #img = resize(img, (IMAGE_SIZE, IMAGE_SIZE), mode='reflect')
            img = img.convert('RGB')
            

            batch_images[i] = preprocess_input(np.array(img, dtype=np.float32)) # Convert to float32 array
            #img.close()

        return batch_images, batch_coords
    

In [ ]:
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i] # Get batch
            pred = self.model.predict_on_batch(batch_images) # Predict bbox on batch

            # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = diff_width * diff_height

            # Compute union
            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = area_gt + area_pred - intersection

            # Compute intersection and union over multiple boxes
            for j, _ in enumerate(union):
                if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
                    intersections += intersection[j]
                    unions += union[j]

        # Compute IOU. Use epsilon to prevent division by zero
        smooth = tf.ones(tf.shape(intersections))
        #tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))
        iou = tf.reduce_mean((intersections+smooth) / (unions - intersections + smooth ))
        logs["val_iou"] = iou

        print(" - val_iou: {}".format(iou))

In [ ]:
train_datagen = DataSequence(train_X)
val_data=Validation(generator=DataSequence(val_X))

In [ ]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * tf.keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred,axis=1)
    union = tf.reduce_sum(y_true,axis=1) + tf.reduce_sum(y_pred,axis=1)
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))


In [ ]:
def create_model(trainable=True):

  model=ResNet50(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),include_top=False)
  for layer in model.layers:
    layer.trainable=trainable
  layer1_br = model.layers[-35].output
  layer2_br= Conv2D(512,(3,3),padding='same',activation='relu')(layer1_br)#Add 2nd FC layer to it 
  #layer2_br=tf.keras.layers.BatchNormalization(momentum=0.9)(layer2_br)
  layer3=Conv2D(512,(1,1))(layer2_br)
  layer4=Conv2D(128,(1,1))(layer3)
  layer4=Conv2D(256,(3,3),strides=(2,2),name='convsame',padding='same',activation='relu')(layer4)#Add 3rd FC layer to it
  #layer4=tf.keras.layers.BatchNormalization(momentum=0.9)(layer4)
  layer5=Conv2D(64,(1,1))(layer4)
  layer5=Conv2D(128,(3,3),strides=(2,2),padding='same',activation='relu')(layer5)#Add 4rth FC layer to it
  layer6=tf.keras.layers.AveragePooling2D((3,3))(layer5)
  layer7=Conv2D(4,(1,1))(layer6)#Add 5th FC layer to it
  R_layer= Reshape((4,))(layer7)
  #flatlayer1_br=tf.keras.layers.Flatten()(layer1_br)
  #flatlayer1_br= tf.keras.layers.Dense(4,activation='relu')(flatlayer1_br)
  #Adding Second Fully Connected Layer
  #flatlayer2_br = tf.keras.layers.Flatten()(layer2_br)
  #flatlayer2_br=  tf.keras.layers.Dense(4,activation='relu')(flatlayer2_br)

  #Adding Third Fully Connected Layer
  #flatlayer4=tf.keras.layers.Flatten()(layer4)
  #flatlayer4=tf.keras.layers.Dense(4,activation='relu')(flatlayer4)

  #Adding Fourth Fully Connected Layer
  #flatlayer5=tf.keras.layers.Flatten()(layer5)
  #flatlayer5=tf.keras.layers.Dense(4,activation='relu')(flatlayer5)

  #Adding Fifth and Final Fully Connected Layer
  #flatlayer7=tf.keras.layers.Flatten()(layer7)
  #flatlayer7=tf.keras.layers.Dense(4,activation='relu')(flatlayer7)

  # Adding Flat Layers

  #Add_Final=tf.keras.layers.concatenate([flatlayer1_br,flatlayer2_br,flatlayer4,flatlayer5,flatlayer7])

  #flatlayer8=tf.keras.layers.Dense(4)(Add_Final)

 
  #model = Model(inputs=visible, outputs=flatlayer8)
  return tf.keras.Model(inputs=model.input, outputs=R_layer)




In [ ]:
model=create_model(False)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
#optimizer=keras.optimizers.SGD(learning_rate=.01,nesterov=True,momentum=.9)
model.compile(loss=iou_bce_loss, optimizer='rmsprop', metrics=[mean_iou])
def cosine_annealing(x):
  lr = 0.001
  epochs = EPOCHS
  return lr*(np.cos(np.pi*x/epochs)+1.)/2

learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)
#model.compile(optimizer='adam', loss=iou_bce_loss,  metrics=['accuracy', mean_iou])
checkpoint = ModelCheckpoint("model-h1vgg1-{val_iou:.2f}.h5", monitor="val_iou", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="max", period=1) # Checkpoint best validation model
stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max") # Stop early, if the validation error deteriorates
reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.2, patience=10, min_lr=1e-5, verbose=1, mode="max")
# Reduce learning rate if Validation IOU does not improve

history=model.fit_generator(generator=train_datagen,
                    epochs=EPOCHS,
                    callbacks=[val_data, checkpoint, reduce_lr, stop,learning_rate],
                    workers=THREADS,
                    use_multiprocessing=MULTI_PROCESSING,
                    shuffle=True,
                    verbose=1)

Epoch 1/15
214/662 [========>.....................] - ETA: 52:31 - loss: 3.1969 - mean_iou: 0.0098

KeyboardInterrupt: ignored

In [ ]:
model=ResNet50()

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.layers[-35].name

'conv4_block6_out'